<h1>Temporal Difference model</h1>

<h2>QUESTIONS</h2>

<ul>
<li>"Thus, component q of the state vector is 1 if the stimulus presentation was exactly q 1 time steps ago and 0 otherwise" ??</li>
<li>r(t) : reward value, quelle(s) valeur(s) peut-elle prendre ?</li>
</ul>

<h3>Discretisation</h3>

In [1]:
T = 5
h = 0.005 #time step scale
N = int(T//h) #time steps occurrence

<h3>Parameters</h3>

In [2]:
gamma = 0.98 #discount factor
alpha = 0.5 #learning rate
lamb = 0.5 #eligibility trace parameter

<h3>Variables initialization</h3>

In [3]:
k = 2 #stimuli occurrence
x = [[0 for t in range(N)] for i in range(k)] #state vectors of the stimuli
w = [0 for t in range(N)] #weights vector
r = [0 for t in range(N)] #reward

r[750] = 1 #reward at time step #750

<h3>Reward prediction</h3>

In [4]:
pl = [[x[i][t]*w[t] for t in range(N)] for i in range(k)] #reward predictions
P = pl[0] #total reward prediction
if k > 1:
    for t in range(N):
        for j in range(1,k): #begin at 1 <-- P = pl[0]
            P[t] += pl[j][t]

<h3>Temporal difference</h3>

In [5]:
TD = [(P[t-1] - gamma*P[t]) for t in range(1, N)]

<h3>Prediction error</h3>

In [6]:
delta = [(r[t] - TD[t]) for t in range(N-1)] #the last time step is ignored

<h3>Eligibility trace</h3>

In [7]:
e = [[0 for t in range(N)] for i in range(k)]

for i in range(k):
    e[i][0] = 0

for t in range(1, N):
    for i in range(k):
        e[i][t] = lamb*e[i][t-1] + x[i][t-1]

<h3>Weight change</h3>

In [8]:
delta_w = [[alpha * delta[t] * e[i][t] for t in range(N-1)] for i in range(k)] #the last time step is ignored